In [1]:
import tensorflow as tf

In [2]:
from modules.data_exploration.data_ex import import_data

In [3]:
data = import_data('data/churn.csv')
display(data)

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,class
0,16,128,415,2845,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,2301,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1616,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,2510,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,155,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,11,50,408,2000,0,1,40,235.7,127,40.07,...,126,18.96,297.5,116,13.39,9.9,5,2.67,2,0
4996,49,152,415,394,0,0,0,184.2,90,31.31,...,73,21.83,213.6,113,9.61,14.7,2,3.97,3,1
4997,7,61,415,313,0,0,0,140.6,89,23.90,...,128,14.69,212.4,97,9.56,13.6,4,3.67,1,0
4998,7,109,510,3471,0,0,0,188.8,67,32.10,...,92,14.59,224.4,89,10.10,8.5,6,2.30,0,0


In [4]:
y_column = 'class'
from sklearn.model_selection import train_test_split
X = data.drop([y_column], axis=1)
y = data[y_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.800)

print('X_train shape:', X_train.shape)
print('y_train shape:',y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (4000, 20)
y_train shape: (4000,)
X_test shape: (1000, 20)
y_test shape: (1000,)


In [5]:
input_shape = len(X_train.columns)


model = tf.keras.models.Sequential() # model instance

# accuracy: 0.8890
#model.add(tf.keras.layers.Dense(20, input_shape=(input_shape,), activation='relu'))
#model.add(tf.keras.layers.Dropout(rate=0.2))
#model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dropout(rate=0.2))

# accuracy: 0.915
#model.add(tf.keras.layers.Dense(20, input_shape=(input_shape,), activation='relu'))
#model.add(tf.keras.layers.Dropout(rate=0.2))
#model.add(tf.keras.layers.Dense(8, activation='relu'))
#model.add(tf.keras.layers.Dropout(rate=0.2))

# accuracy: xx
model.add(tf.keras.layers.Dense(20, input_shape=(input_shape,), activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(4, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.2))

model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # output layer with 10 classes to clasify
model.summary() # architecture

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                672       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
_________________________________________________________________
dropout_2 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5

In [6]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.002),
    metrics=['accuracy'])

In [ ]:
epo = 300
history = model.fit(X_train, y_train, epochs=epo, validation_split=0.2)

Epoch 1/300
100/100 [==============================] - 0s 3ms/step - loss: 35.7787 - accuracy: 0.7694 - val_loss: 0.6211 - val_accuracy: 0.8737
Epoch 2/300
100/100 [==============================] - 0s 2ms/step - loss: 1.7535 - accuracy: 0.8494 - val_loss: 0.5857 - val_accuracy: 0.8737
Epoch 3/300
100/100 [==============================] - 0s 2ms/step - loss: 0.8530 - accuracy: 0.8512 - val_loss: 0.5114 - val_accuracy: 0.8737
Epoch 4/300
100/100 [==============================] - 0s 2ms/step - loss: 0.6893 - accuracy: 0.8525 - val_loss: 0.4791 - val_accuracy: 0.8737
Epoch 5/300
100/100 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.8531 - val_loss: 0.4434 - val_accuracy: 0.8737
Epoch 6/300
100/100 [==============================] - 0s 2ms/step - loss: 0.5005 - accuracy: 0.8541 - val_loss: 0.4558 - val_accuracy: 0.8737
Epoch 7/300
100/100 [==============================] - 0s 2ms/step - loss: 0.5921 - accuracy: 0.8544 - val_loss: 0.3992 - val_accuracy: 0.873

Epoch 58/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 59/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3806 - val_accuracy: 0.8737
Epoch 60/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3808 - val_accuracy: 0.8737
Epoch 61/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4216 - accuracy: 0.8544 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 62/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3806 - val_accuracy: 0.8737
Epoch 63/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 64/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy:

Epoch 115/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3808 - val_accuracy: 0.8737
Epoch 116/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 117/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3806 - val_accuracy: 0.8737
Epoch 118/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 119/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3808 - val_accuracy: 0.8737
Epoch 120/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 121/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3807 - val_ac

100/100 [==============================] - 0s 2ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3806 - val_accuracy: 0.8737
Epoch 172/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 173/300
100/100 [==============================] - 0s 2ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 174/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3808 - val_accuracy: 0.8737
Epoch 175/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4145 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 176/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3807 - val_accuracy: 0.8737
Epoch 177/300
100/100 [==============================] - 0s 1ms/step - loss: 0.4146 - accuracy: 0.8547 - val_loss: 0.3806 - val_accuracy: 0.8737

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='train set accuracy')
plt.plot(history.history['val_accuracy'], label='validation set accuracy')
plt.xticks(range(epo))
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
import numpy as np

y_pred = np.where(y_pred<=0.5, 0, 1)



In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns

data = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(data, columns=['no churn', 'churn'], index=['no churn', 'churn'])
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize=(9, 6))
sns.set(font_scale=1.4)  # for label size
sns.heatmap(df_cm, cmap='RdYlGn', annot=True, annot_kws={"size": 14}, fmt='d')
plt.title("Confusion Matrix", fontweight='bold')
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')
plt.show()

In [ ]:
data